In [ ]:
#data preprocessing pipeline (possible to adapt to other airports):

#flight data:
#1. read the sdr and opensky csv files
#2. change the header names in opensky to match the sdr 
#3. unit conversions (altitude=m, speed=m/s, vertical rate=m/s)
#4. limit the data between the lat and lon of LPPt (excludes helicopters and other smaller planes)
#5. drop duplicates, nans and any outliers (negative altitudes or speeds)
#6. segment the flights (landings, take offs and ground => only the landings stay and each of them have an unique landing index)
#7. merge the sdr and opensky datasets 

#weather data (METAR and ECMWF): 
#1. read the METAR txt file
#2. parse all the relevant features from METAR
#3. calculte new weather features 
#4. match the closest METAR to each flight 
#5. read the ECMWF csv file (+ cat.csv which has additional ECMWF features)
#6. match the closest ECMWF to each flight (the closest ECMWF, the ECMWF before and the ECMWF after are put together and depending on the variable the max or min value is chosen)

#feature engineering pt1:
#1. correct the barometric altitude 
#2. assign each flight a runway (02 or 20)
#3. calculate the distance to the runway
 
#go around label: 
#1. go around algorithm (1 if it is a ga and 0 if it is not)

#feature engineering pt2: 
#1. cut dataset (11NM to when a ga starts or before a normal flight lands, all the flights without info before a ga are taken out)
#2. calculate new flight features (glideslope, localizer deviation and energy)
#3. calculate go around clustering effects (ga count hourly, ga last time, ga before)
#4. calculate in trail relationship features (loss of separation, speed and altitude difference, lead aircraft)

#### flight data:

In [ ]:
#read the sdr and opensky csv files

import pandas as pd
opensky_df = pd.read_csv('opensky_abr24_data.csv')

In [ ]:
#change the header names in opensky to match the sdr 
#unit conversions (altitude=m, speed=m/s, vertical rate=m/s)

opensky_df['datetime'] = pd.to_datetime(opensky_df['time'], unit='s')
opensky_df['Date'] = opensky_df['datetime'].dt.date.astype(str)
opensky_df['Time'] = opensky_df['datetime'].dt.time.astype(str)
opensky_df['Time_Sec'] = opensky_df['datetime'].dt.hour * 3600 + opensky_df['datetime'].dt.minute * 60 + opensky_df['datetime'].dt.second

opensky_df = opensky_df.rename(columns={
    "icao24": "ICAO",
    "callsign":"Callsign",
    "lat": "Latitude",
    "lon": "Longitude",
    "baroaltitude": "Altitude",
    "velocity": "Speed",
    "vertrate": "Vertical Rate",
    "heading": "Heading"})

In [ ]:
#limit the data between the lat and lon of the LPPT (excludes helicopters and other smaller planes)

#sdr_df = sdr_df[(sdr_df['Latitude'].between(38.6, 38.9)) & (sdr_df['Longitude'].between(-9.2, -9.0))]
opensky_df = opensky_df[(opensky_df['Latitude'].between(38.6, 38.9)) & (opensky_df['Longitude'].between(-9.2, -9.0))]

In [ ]:
#drop duplicates, nans and any outliers (negative altitudes or speeds)

required_columns = ["Latitude", "Longitude", "Altitude", "Speed", "Vertical Rate", "Heading"]

cleaned_opensky = []
for (icao, callsign, date), group in opensky_df.groupby(["ICAO", "Callsign", "Date"]):
    group_clean = group.dropna(subset=required_columns)
    if len(group_clean) > 0:
        cleaned_opensky.append(group_clean)
opensky_df = pd.concat(cleaned_opensky, ignore_index=True)

opensky_df = opensky_df.drop(opensky_df[(opensky_df.Altitude < 0) | (opensky_df.Altitude > 2000)].index)
opensky_df = opensky_df.drop(opensky_df[(opensky_df.Speed < 0) | (opensky_df.Speed > 250)].index)

opensky_df["Date"] = pd.to_datetime(opensky_df["Date"])
opensky_df = opensky_df[opensky_df["Date"].dt.month == 4]

features = ["Latitude", "Longitude", "Speed", "Vertical Rate", "Heading", "Altitude"]
n_features = len(features)  
model_df = opensky_df

for feature in features:
    if feature in model_df.columns:
        values = model_df[feature].dropna() 
        mean = values.mean()
        std = values.std()
        min_val = values.min()
        max_val = values.max()
        print(f"{feature}: mean={mean:.2f}, std={std:.2f}, min={min_val:.2f}, max={max_val:.2f}")
    else:
        print(f"{feature}: not found in model_df")

In [ ]:
#segment the flights (landings, take offs and ground => only the landings stay and each of them have an unique landing index)

##########################################################################################################################################################
#how it works:
#1. remove rows with missing callsigns and callsigns from helicopters and small flights

#2. organize the data per ICAO, callsign and date and sort it per choronological order (represents all the flights an aircraft made per day)

#3. for each aircraft, if the time difference between two consecutive timestamps is bigger than 1800s (30 min), the data is cut and saved to a list (represents when an aircraft stopped and started a new flight => fligth phase transitions)

#4. for each flight, the flight phase is found (C = ground/cruise, L = landing and T = take-off)
    #if duration > 600 and altitude change < 50:
        #flight phase = C 
    #if altitude min > 3000 or altitude max < 500: 
        #flight phase = C
    #if there are static sequences in altitude, speed or vertical rate:
        #flight phase = C
    #if average vertical rate < 0:
        #flight phase = L
    #else if average vertical rate > 0:
        #flight phase = T
    #else: 
        #flight phase = C

#5. create dataset with only the landings (each flight has a different landing index)
#########################################################################################################################################################

#function: converts the data to numeric
def prepare_data(df):
    numeric_cols = ["Latitude", "Longitude", "Altitude", "Speed", "Vertical Rate", "Heading"]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    return df

#function: segments the data in order to create a dataset with the landings
def segment_landings(df, time_col="Time_Sec", min_msgs=100, threshold=10):
    landing_index = 0
    all_landing_data = []

    df = df[df['Callsign'].notna()].copy() 
    df['Callsign'] = df['Callsign'].astype(str).str.strip().str.upper() 
    df = df[~df['Callsign'].str.startswith(('G', 'L', 'H', 'EC', 'BB', 'BS', 'IFJ', 'BAE', 'JME', 'N', 'VJH', 'KAF', 'OCEAN', 'BLAST', 'JFA', 'ANTENA'))]

    for _, data in df.groupby(["ICAO", "Callsign", "Date"]):
        data = data.sort_values(by=time_col).reset_index(drop=True)
        data['Time_Diff'] = data[time_col].diff()

        phase_change = data.index[data["Time_Diff"] > 1800].tolist() 
        phase_change.insert(0, 0)
        phase_change.append(len(data))

        phase_attribute = ["Unknown"] * len(data)
        
        for flight_seg in range(len(phase_change) - 1):
            start, end = phase_change[flight_seg], phase_change[flight_seg + 1]
            phase_seg = data.iloc[start:end].copy()
            vertrate = phase_seg["Vertical Rate"].tail(min(200, len(phase_seg)))
            avg_vertrate = vertrate.mean()

            skip_landing_flag = False

            altitude_change = phase_seg["Altitude"].iloc[-1] - phase_seg["Altitude"].iloc[0]
            duration = phase_seg[time_col].iloc[-1] - phase_seg[time_col].iloc[0]

            if duration > 600 and abs(altitude_change) < 50:
                phase_attribute[start:end] = ["C"] * (end - start) 
                skip_landing_flag = True
                continue  

            altitude_tolerance = 5  
            speed_tolerance = 3 
            vertrate_tolerance = 0.5 

            count = 0
            for i in range(1, len(phase_seg)): 
                if (abs(phase_seg["Altitude"].iloc[i] - phase_seg["Altitude"].iloc[i - 1]) <= altitude_tolerance and
                    abs(phase_seg["Speed"].iloc[i] - phase_seg["Speed"].iloc[i - 1]) <= speed_tolerance and
                    abs(phase_seg["Vertical Rate"].iloc[i] - phase_seg["Vertical Rate"].iloc[i - 1]) <= vertrate_tolerance):
                    count += 1
                    final_vertrate = phase_seg["Vertical Rate"].iloc[-1]

                    if count >= threshold:
                        if final_vertrate < 0.5:
                            count = 0
                        else:
                            phase_attribute[start:end] = ["C"] * (end - start)
                            skip_landing_flag = True
                            break
                else:
                    count = 0 

            if skip_landing_flag:
                continue
            
            if phase_seg["Altitude"].min() > 3000:
                continue

            if phase_seg["Altitude"].max() < 500:
                continue

            if avg_vertrate < 0 and len(phase_seg) >= min_msgs and not skip_landing_flag:
                phase_seg = phase_seg.copy()
                phase_seg["Phase of Flight"] = "L"
                phase_seg["Landing_Index"] = landing_index
                all_landing_data.append(phase_seg)
                landing_index += 1
            else:
                phase_type = "T" if avg_vertrate > 0 else "C"
                phase_attribute[start:end] = [phase_type] * (end - start)

        data["Phase of Flight"] = phase_attribute

    if all_landing_data:
        return pd.concat(all_landing_data, ignore_index=True)
    else:
        return pd.DataFrame()
    
opensky_df = prepare_data(opensky_df)
openskylandings_df = segment_landings(opensky_df)

In [ ]:
#check to confirm values
features = ["Latitude", "Longitude", "Speed", "Vertical Rate", "Heading", "Altitude"]

n_features = len(features)  
model_df = openskylandings_df

for feature in features:
    if feature in model_df.columns:
        values = model_df[feature].dropna() 
        mean = values.mean()
        std = values.std()
        min_val = values.min()
        max_val = values.max()
        print(f"{feature}: mean={mean:.2f}, std={std:.2f}, min={min_val:.2f}, max={max_val:.2f}")
    else:
        print(f"{feature}: not found in model_df")

In [ ]:
#merge the sdr and opensky datasets 

merged_df = openskylandings_df
merged_df = merged_df.drop_duplicates(subset=['Date', 'Time', 'Time_Sec','ICAO', 'Callsign', 'Latitude', 'Longitude', 'Speed', 'Vertical Rate', 'Altitude', 'Heading'])
merged_df = merged_df.sort_values(by=["ICAO", "Callsign", "Date", "Time_Sec"]).reset_index(drop=True)
merged_df['Landing_Index'] = merged_df.groupby(["ICAO", "Callsign", "Date"]).ngroup() + 1 
merged_df = merged_df.drop(columns=['Time_Diff','Phase of Flight'])
desired_order = [
    'Date', 'Time', 'Time_Sec',
    'ICAO', 'Callsign',
    'Latitude', 'Longitude',
    'Speed', 'Vertical Rate', 'Altitude', 'Heading',
    'Landing_Index'
]
merged_df = merged_df[[col for col in desired_order if col in merged_df.columns]]
merged_df.to_csv("april_data.csv", mode='a', header=True, index=False)

In [ ]:
#check the data

flight_dataset = pd.read_csv('april_data.csv', low_memory=False)
flight_dataset

#### weather data: METAR

In [ ]:
#read the METAR txt file
#parse all the relevant features from METAR

import re
import csv
from datetime import datetime
import pandas as pd

def parse_metar_row(metar_str, year=2024, month=4): #change the month and the year
    
    global last_wind_dir

    time_match = re.search(r"\b(\d{2})(\d{2})(\d{2})Z\b", metar_str)
    if not time_match:
        print("Time not found in:", metar_str)
        return None
    
    day, hour, minute = map(int, time_match.groups())
    timestamp = datetime(year, month, day, hour, minute)

    temp_match = re.search(r"(\d{2})/(\d{2})", metar_str)
    temp = int(temp_match.group(1)) if temp_match else None

    pressure_match = re.search(r"Q(\d{4})", metar_str)
    pressure = int(pressure_match.group(1)) if pressure_match else None

    wind_match = re.search(r"(VRB|\d{3})(\d{2})G?(\d{2})?KT", metar_str)

    if wind_match:
        try:
            direction_str = wind_match.group(1)
            if direction_str == "VRB":
                wind_direction = last_wind_dir 
            else:
                wind_direction = int(direction_str)
                last_wind_dir = wind_direction

            wind_speed_kt = int(wind_match.group(2))
            wind_speed = wind_speed_kt * 0.514444444
            wind_gust = int(bool(wind_match.group(3)))
        except:
            wind_direction = None
            wind_speed = None
            wind_gust = 0
    else:
        wind_direction = None
        wind_speed = None
        wind_gust = 0
    
    thunderstorm = int(bool(re.search(r"\bTS\b", metar_str)))
    fog = int(bool(re.search(r"\bFG\b", metar_str)))
    rain = int(bool(re.search(r"\bRA\b", metar_str)))
    wind_shear = int(bool(re.search(r"\bWS\b", metar_str)))
    cumo = int(bool(re.search(r"\bCB\b", metar_str)))
    tcu = int(bool(re.search(r"\bTCU\b", metar_str)))
    cb = 0
    if cumo == 1 or tcu == 1: 
        cb = 1

    vis_match = re.search(r"\b(\d{4})\b", metar_str)
    visibility = int(vis_match.group(1)) if vis_match else None

    return {
        "Timestamp": timestamp,
        "Temperature_M": temp,
        "Pressure_M": pressure,
        "Wind_Speed_M": wind_speed,
        "Wind_Direction_M": wind_direction,
        "Visibility_M": visibility,
        "G_M": wind_gust,
        "TS_M": thunderstorm,
        "FG_M": fog,
        "RA_M": rain,
        "WS_M": wind_shear,
        "CB_M": cb
    }

output_file = "april_metars.csv"
with open(output_file, "w", newline="") as out_file:
    fieldnames = ["Timestamp", "Temperature_M", "Pressure_M", "Wind_Speed_M", "Wind_Direction_M", "Visibility_M", "G_M", "TS_M", "FG_M", "RA_M", "WS_M", "CB_M"]
    writer = csv.DictWriter(out_file, fieldnames=fieldnames)
    writer.writeheader()

    with open("metar.202404", "r") as file: #change the month and year
        for line in file:
            metar = line.strip().replace("METAR ", "")
            if metar:
                try:
                    row = parse_metar_row(metar)
                    if row:
                        writer.writerow(row)
                except Exception as e:
                    print(f"error")
                    print("Line that caused error:", metar)

metars_df = pd.read_csv(output_file, parse_dates=["Timestamp"])

In [ ]:
#calculte new weather features 

import pandas as pd
import numpy as np

def calc_wind_dir_change(wind_dir_m, wind_speed_m, threshold=2.0):
    wind_dir = pd.to_numeric(wind_dir_m, errors='coerce')
    wind_speed = pd.to_numeric(wind_speed_m, errors='coerce')

    valid = (
        wind_dir.notna() & wind_dir.shift(1).notna() &
        wind_speed.notna() & wind_speed.shift(1).notna())

    diff = wind_dir.diff().abs()
    diff = np.where(diff > 180, 360 - diff, diff)

    change = pd.Series(np.nan, index=wind_dir.index, dtype=float) 
    change[valid] = diff[valid]

    low_speed_mask = (wind_speed < threshold) | (wind_speed.shift(1) < threshold)
    change[valid & low_speed_mask] = np.nan
    change.iloc[0] = np.nan

    change_cat = pd.Series(np.nan, index=change.index, dtype=float)
    change_cat[change < 30] = 0
    change_cat[change >= 30] = 1
    change_cat.fillna(0, inplace=True)

    return change, change_cat

def calc_wind_dir_sector(wind_dir_m, wind_speed_m, wind_dir_change_m, decl=23.0, threshold=2.0):
    wd = pd.to_numeric(wind_dir_m, errors='coerce')
    ws = pd.to_numeric(wind_speed_m, errors='coerce')
    wdc = pd.to_numeric(wind_dir_change_m, errors='coerce')

    valid = (
        wd.notna() & wd.shift(1).notna() &
        ws.notna() & ws.shift(1).notna() &
        wdc.notna())

    area_A = ((wd >= (90 + decl)) & (wd < (270 + decl)))
    area_B = ((wd >= (270 + decl)) & (wd <= 360)) | (wd < (90 + decl))

    prev_wd = wd.shift(1)
    prev_area_A = ((prev_wd >= (90 + decl)) & (prev_wd < (270 + decl)))
    prev_area_B = ((prev_wd >= (270 + decl)) & (prev_wd <= 360)) | (prev_wd < (90 + decl))

    cross_area = pd.Series(np.nan, index=wd.index, dtype=float)
    same_area = ((area_A & prev_area_A) | (area_B & prev_area_B))

    shear_condition = (wdc >= 10) | ((wdc >= 7.5) & (ws >= 4))
    cross_area[valid & shear_condition & same_area] = 0
    cross_area[valid & shear_condition & ~same_area] = 1

    low_speed_mask = (ws < threshold) | (ws.shift(1) < threshold)
    exception_mask = (ws.shift(1) < threshold) & (ws >= 2 * threshold)
    cross_area[valid & shear_condition & low_speed_mask & ~exception_mask] = np.nan

    cross_area.iloc[0] = np.nan
    cross_area.fillna(0, inplace=True)

    return cross_area

wind_dir_change, wind_dir_change_cat = calc_wind_dir_change(metars_df["Wind_Direction_M"], metars_df["Wind_Speed_M"])

metars_df["Wind_Dir_Change"] = wind_dir_change
metars_df["Wind_Dir_Change_M"] = wind_dir_change_cat

metars_df["Wind_Dir_Sector_M"] = calc_wind_dir_sector(metars_df["Wind_Direction_M"], metars_df["Wind_Speed_M"], metars_df["Wind_Dir_Change"])

In [ ]:
#match the closest METAR to each flight 

flight_dataset["Timestamp"] = pd.to_datetime(flight_dataset["Date"] + " " + flight_dataset["Time"])
landing_times = flight_dataset.groupby("Landing_Index")["Timestamp"].max().reset_index() 

#find the closest metar
def find_closest_metar(landing_time):
    time_metar = (metars_df["Timestamp"] - landing_time).abs()
    closest_idx = time_metar.idxmin()
    window_df = metars_df.loc[closest_idx].copy()

    result = {}

    for col in ["Temperature_M", "Pressure_M", "Wind_Speed_M", "Wind_Direction_M", "Visibility_M", "G_M", "TS_M", "FG_M", "RA_M", "WS_M", "CB_M", 'Wind_Dir_Change', 'Wind_Dir_Change_M', 'Wind_Dir_Sector_M']:
        result[col] = window_df[col]

    result["Timestamp"] = landing_time
    return pd.Series(result)

closest_metars = landing_times["Timestamp"].apply(find_closest_metar)
closest_metars = pd.concat([landing_times["Landing_Index"].reset_index(drop=True), closest_metars.reset_index(drop=True)], axis=1)

flight_metar_dataset = flight_dataset.merge(closest_metars, on="Landing_Index", how="left")
flight_metar_dataset = flight_metar_dataset.drop(columns=['Timestamp_x','Timestamp_y'])
flight_metar_dataset

#### weather data: ECMWF

In [ ]:
#read the ECMWF csv file (+ cat.csv which has additional ECMWF features) 

import pandas as pd
from datetime import datetime

input_file = "ecmwf_mar_mai_2024.csv"
output_file = "ecmwf.csv"

final_columns = [
    "Timestamp", "Visibility_E", "Wind_CompU_E", "Wind_CompV_E", "Wind_Gust_E",
    "Wind_Shear_E", "CBHA_E", "Wind_Direction_E"]

data = []

with open(input_file, "r") as file:
    for idx, line in enumerate(file):
        parts = line.strip().split()
        if len(parts) != 14:
            print(f"[Line {idx}] Skipping malformed line with {len(parts)} columns: {line}")
            continue
        try:
            timestamp_str = parts[0]
            timestamp = datetime.strptime(timestamp_str, "%Y%m%d%H")
        except ValueError as e:
            print(f"[Line {idx}] Invalid date format: {timestamp_str}")
            continue
        try:
            visibility = float(parts[3]) 
            if visibility < 0:
                visibility = None
            wind_compu = float(parts[5])
            wind_compv = float(parts[6])
            wind_gust = float(parts[7])
            wind_10m = float(parts[4])
            wind_925 = float(parts[9])
            cbha = float(parts[11])
            wind_shear = wind_925 - wind_10m
            wind_dir_10m = float(parts[13])

            row = [
                timestamp,
                visibility,
                wind_compu,
                wind_compv,
                wind_gust,
                wind_shear,
                cbha,
                wind_dir_10m
            ]

            data.append(row)

        except ValueError:
            print(f"[Line {idx}] Failed to convert values to float: {parts[1:]}")
            continue

ecmwf_df = pd.DataFrame(data, columns=final_columns)
ecmwf_df.to_csv(output_file, index=False)


In [ ]:
#match the closest ECMWF to each flight (the closest ECMWF, the ECMWF before and the ECMWF after are put together and depending on the variable the max or min value is chosen)

flight_metar_dataset["Timestamp"] = pd.to_datetime(flight_metar_dataset["Date"] + " " + flight_metar_dataset["Time"])
landing_times = flight_metar_dataset.groupby("Landing_Index")["Timestamp"].max().reset_index()

def find_closest_ecmwf(landing_time):
    time_ecmwf = (ecmwf_df["Timestamp"] - landing_time).abs()
    closest_idx = time_ecmwf.idxmin()
    
    idxs = [i for i in [closest_idx - 1, closest_idx, closest_idx + 1] if i in ecmwf_df.index] 
    window_df = ecmwf_df.loc[idxs].copy()

    result = {}

    for col in ["Wind_CompU_E", "Wind_CompV_E", "Wind_Gust_E", "Wind_Shear_E", "CBHA_E", "Wind_Direction_E"]:
        result[col] = window_df[col].max()

    for col in ["Visibility_E"]:
        result[col] = window_df[col].min()

    result["Timestamp"] = landing_time
    return pd.Series(result)

closest_ecmwf = landing_times["Timestamp"].apply(find_closest_ecmwf)
closest_ecmwf = pd.concat([landing_times["Landing_Index"].reset_index(drop=True), closest_ecmwf.reset_index(drop=True)], axis=1)

full_dataset = flight_metar_dataset.merge(closest_ecmwf, on="Landing_Index", how="left")
full_dataset = full_dataset.drop(columns=['Timestamp_x','Timestamp_y']) #decidir se retiro mais ou nao

In [ ]:
#read the ECMWF csv file (+ cat.csv which has additional ECMWF features)

import pandas as pd
from datetime import datetime

input_file = "cat_timeseries.csv"
output_file = "cat.csv"

final_columns = ["Timestamp", "C_1000_E", "C_500_E"]

data = []

with open(input_file, "r") as file:
    for idx, line in enumerate(file):
        parts = line.strip().split(",")
        if len(parts) != 3:
            print(f"[Line {idx}] Skipping malformed line with {len(parts)} columns: {line}")
            continue
        try:
            timestamp_str = parts[0]
            timestamp = datetime.strptime(timestamp_str, "%Y%m%d%H")
        except ValueError as e:
            print(f"[Line {idx}] Invalid date format: {timestamp_str}")
            continue
        try:
            c1000 = float(parts[1]) 
            c500 = float(parts[2])

            row = [
                timestamp,
                c1000,
                c500]
            data.append(row)

        except ValueError:
            print(f"[Line {idx}] Failed to convert values to float: {parts[1:]}")
            continue

cat_df = pd.DataFrame(data, columns=final_columns)
cat_df.to_csv(output_file, index=False)

In [ ]:
#match the closest ECMWF to each flight 

full_dataset["Timestamp"] = pd.to_datetime(full_dataset["Date"] + " " + full_dataset["Time"])
landing_times = full_dataset.groupby("Landing_Index")["Timestamp"].max().reset_index()

def find_closest(landing_time):
    time = (cat_df["Timestamp"] - landing_time).abs()
    closest_idx = time.idxmin()
    
    idxs = [i for i in [closest_idx - 1, closest_idx, closest_idx + 1] if i in cat_df.index] #vou buscar os indexes antes e depois
    window_df = cat_df.loc[idxs].copy()

    result = {}

    for col in ["C_1000_E", "C_500_E"]:
        result[col] = window_df[col].max()

    result["Timestamp"] = landing_time
    return pd.Series(result)

closest = landing_times["Timestamp"].apply(find_closest)
closest = pd.concat([landing_times["Landing_Index"].reset_index(drop=True), closest.reset_index(drop=True)], axis=1)

all_dataset = full_dataset.merge(closest, on="Landing_Index", how="left")
all_dataset = all_dataset.drop(columns=['Timestamp_x','Timestamp_y']) 
all_dataset.replace(-999, np.nan, inplace=True)
all_dataset.to_csv("april.csv", index=False)

In [ ]:
#check to confirm the values

features = ["Wind_Speed_M", "Wind_Direction_M", "Visibility_M", "G_M", "TS_M", "FG_M", "RA_M", "WS_M", "Wind_Dir_Change", "Wind_Dir_Change_M", "Wind_Dir_Sector_M",
            "Wind_CompU_E", "Wind_CompV_E", "Wind_Gust_E", "Wind_Shear_E", "CBHA_E", "Wind_Direction_E", "Visibility_E", "C_1000_E", "C_500_E"]

n_features = len(features)  
model_df = all_dataset

for feature in features:
    if feature in model_df.columns:
        values = model_df[feature].dropna() 
        mean = values.mean()
        std = values.std()
        min_val = values.min()
        max_val = values.max()
        print(f"{feature}: mean={mean:.2f}, std={std:.2f}, min={min_val:.2f}, max={max_val:.2f}")
    else:
        print(f"{feature}: not found in model_df")

In [ ]:
#check the data

dataset = pd.read_csv("april.csv", dtype={"ICAO": str})
dataset

#### feature engineering part 1:

In [ ]:
#correct the barometric altitude 

def correct_altitude(temp, pressure, altitude):
    if pd.isna(temp) or pd.isna(pressure) or pd.isna(altitude):
        return None
    T_metar = temp + 273.15 #celsius to kelvin
    P_metar = pressure #hPa
    Alt = altitude #meters
    T_isa = 288.15 #K
    P_isa = 1013.25 #hPa

    try:
        T_a = (P_isa*(1-0.0065*(Alt/T_isa))**5.2561)/P_metar
        real_altitude_ft = ((3.28084*T_metar)*(1-(T_a)**0.190255))/0.0065
        real_altitude = real_altitude_ft*0.3048
        return real_altitude
    except:
        return None

dataset["Correct Altitude"] = dataset.apply(lambda row: correct_altitude(row["Temperature_M"], row["Pressure_M"], row["Altitude"]),axis=1)

In [ ]:
#assign each flight a runway (02 or 20)

def runway(group):
    group = group.drop(columns=["Landing_Index"], errors="ignore")
    
    if len(group) < 10:
        return pd.Series({"Runway": np.nan})
    
    heading_start = group["Heading"].iloc[0]
    rwy = "02" if abs(heading_start - 22.72) < abs(heading_start - 202.73) else "20"
    return pd.Series({"Runway": rwy})

runway_df = (
    dataset.groupby("Landing_Index")
    .apply(runway, include_groups = False)
    .reset_index())
dataset = dataset.merge(runway_df, on="Landing_Index", how="left")

In [ ]:
#calculate the distance to the runway

from geopy.distance import geodesic

def calc_distance(latitude, longitude, runway):
    if runway == '02':
        runway_points = (38.76638889,-9.14388889)
    else:
        runway_points = (38.79222222,-9.13000000)
    distance = geodesic((latitude, longitude), runway_points).meters
    return distance

dataset["Distance"] = dataset.apply(lambda row: calc_distance(row["Latitude"], row["Longitude"], row['Runway']), axis=1)

#### go around label:

In [ ]:
#go around algorithm (1 if it is a ga and 0 if it is not)

##########################################################################################################################################################
#how it works:
#1. organize the flights per landing index and sort it per chronological order

#2. if a flight has less than 30 altitude values:
        #ga label = 0

#3. if the max vertical rate is less than 8 m/s:
        #ga label = 0

#4. calculate the difference between consecutive altitudes, the index where the altitudes difference is bigger than the altitude threshold (90 m) is saved

#5. if the altitude difference >= 90 m:
        #if unrealistic jumps = true or horizontal jumps = true:
            #apply rdp algorithm
            #if trajectory intersects with itself (no runway change):
                #ga label = 1
            #else aircraft heading change (runway change + different heading):
                #ga label = 1
            #else 2 or more heading changes (runway change + similar heading):
                #ga label = 1
            #else:
                #ga label = 0 
        #else:
            #ga label = 0 or errors      

#6: if gradual climb = true:
        #ga label = 1
   #else:
        #ga label = 0

#5. add the go around label to the existing dataset
#########################################################################################################################################################

import pandas as pd
import numpy as np
from haversine import haversine, Unit
from shapely.geometry import LineString
from rdp import rdp

#function: checks if there was any gradual climb (if there is two or more 50 m climbs)
def gradual_climb(group, window=10, min_gain=50, min_climbs=2): 
    altitudes = group["Altitude"].values
    count = 0
    for i in range(len(altitudes) - window):
        gain = altitudes[i + window] - altitudes[i]
        if gain >= min_gain:
            count += 1
        if count >= min_climbs:
            return 1
    return 0

#function: detects if a flight performed or not a go around
def detect_ga(df, altitude_threshold=90, turning_threshold=2):
    results = []
    early_discarded = 0  

    for landing_index, data in df.groupby("Landing_Index"):
        data = data.sort_values("Time_Sec").reset_index(drop=True)

        altitudes = data["Altitude"].values
        vertrates = data["Vertical Rate"].values
        coordinates = data[["Latitude", "Longitude"]].values

        if len(altitudes) < 30:
            results.append((landing_index, 0))
            continue

        max_vertrate = np.max(vertrates)
        if max_vertrate < 8:
            early_discarded += 1 
            results.append((landing_index, 0))
            continue

        altitude_diff = np.diff(altitudes)
        altitude_climb = np.where(altitude_diff > altitude_threshold)[0]

        if len(altitude_climb) > 0:
            climb_idx = altitude_climb[0]
            trajectory_afterclimb = coordinates[climb_idx:]

            if climb_idx + 1 < len(data):
                altitude_now = data.loc[climb_idx, "Altitude"]
                altitude_next = data.loc[climb_idx + 1, "Altitude"]
                jump = abs(altitude_next - altitude_now)

                lat1, lon1 = data.loc[climb_idx, ["Latitude", "Longitude"]]
                coordinates1 = (lat1, lon1)
                lat2, lon2 = data.loc[climb_idx + 1, ["Latitude", "Longitude"]]
                coordinates2 = (lat2, lon2)
                dist = haversine(coordinates1, coordinates2, unit=Unit.METERS)

                if jump > 1000 or dist > 7000:
                    print(f"Skipping Landing_Index {landing_index} — jump {jump:.0f} m, distance {dist:.1f} m")
                    results.append((landing_index, 0))
                    continue

            trajectory_line = LineString(trajectory_afterclimb)
            if trajectory_line.crosses(LineString(trajectory_afterclimb)):
                results.append((landing_index, 1))
                continue

            if "Track Angle" in data.columns:
                if climb_idx + 1 < len(data):  
                    heading_start = data.loc[climb_idx, "Track Angle"]
                    heading_end = data.loc[data.index[-1], "Track Angle"]

                    heading_diff = abs(heading_end - heading_start)
                    heading_diff = 360 - heading_diff if heading_diff > 180 else heading_diff

                    if heading_diff > 30:
                        results.append((landing_index, 1))
                        continue

            trajectory_simplified = rdp(trajectory_afterclimb, epsilon=0.0005)
            if len(trajectory_simplified) - 2 > turning_threshold:
                results.append((landing_index, 1))
                continue

        gradual = gradual_climb(data, window=10, min_gain=50, min_climbs=2)
        if gradual == 0: 
             print(f"Skipping Landing_Index {landing_index} — no gradual ascend")
        results.append((landing_index, 1 if gradual else 0))

    print(f"Total flights discarded early: {early_discarded}")     
    return pd.DataFrame(results, columns=["Landing_Index", "Go_Around_Label"])

In [ ]:
#go around algorithm (1 if it is a ga and 0 if it is not)

go_around_labels = detect_ga(dataset)
df = dataset.merge(go_around_labels, on="Landing_Index", how="left")

go_around_landings = df[df["Go_Around_Label"] == 1]["Landing_Index"].nunique()
total_landings = df["Landing_Index"].nunique()

print(f"Detected go-arounds: {go_around_landings} out of {total_landings} landings")

go_around_df = df[df["Go_Around_Label"] == 1]
go_around_summary = go_around_df.groupby("Landing_Index").first().reset_index()

go_around_summary["Callsign"] = go_around_summary["Callsign"].fillna("UNKNOWN")
go_around_summary["Date"] = go_around_summary["Date"].fillna("UNKNOWN")

info_cols = ["Landing_Index", "ICAO", "Callsign", "Date", "Time"]
go_around_info = go_around_summary[info_cols]

print(go_around_info)

In [ ]:
print(df["Go_Around_Label"].unique())

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
import contextily as ctx

oknib_lon, oknib_lat = -9.17861111, 38.70222222
lis_lon, lis_lat = -9.134167, 38.774167
ist_lon, ist_lat = -9.137961014549424, 38.73990015557909

go_around_df = df[df['Go_Around_Label'] == 1]
unique_index = go_around_df['Landing_Index'].unique()[:15]  

for callsign in unique_index:
    flight_data = go_around_df[go_around_df['Landing_Index'] == callsign]

    if flight_data.empty:
        continue

    fig, axes = plt.subplots(2, 1, figsize=(10, 8))

    axes[0].plot(flight_data['Time_Sec'], flight_data['Altitude'], 'b-', label="Altitude")
    axes[0].scatter(flight_data['Time_Sec'], flight_data['Altitude'], c='blue', s=5)
    axes[0].set_xlabel('Time')
    #axes[0].set_xlim(0, 1200)
    axes[0].set_ylabel('Altitude (m)')
    axes[0].set_ylim(0, )
    axes[0].grid(True)
    axes[0].legend()

    axes[1].plot(flight_data['Time_Sec'], flight_data['Speed'], 'r-', label="Velocity")
    axes[1].scatter(flight_data['Time_Sec'], flight_data['Speed'], c='red', s=5)
    axes[1].set_xlabel('Time')
    axes[1].set_ylabel('Velocity (m/s)')
    axes[1].set_ylim(0, 200)
    axes[1].grid(True)
    axes[1].legend()

    plt.suptitle(f"Flight Data for Go-Around: {callsign}")
    plt.tight_layout()
    plt.show()

    fig, ax = plt.subplots(figsize=(10, 8))

    gdf = gpd.GeoDataFrame(
        geometry=[Point(lon, lat) for lon, lat in zip(flight_data['Longitude'], flight_data['Latitude'])],
        crs="EPSG:4326"
    )

    gdf.plot(ax=ax, color='blue', linewidth=1, marker='o', markersize=5, label="Trajectory")

    ax.scatter(ist_lon, ist_lat, color='red', s=100, edgecolor='black', label="IST")
    ax.scatter(oknib_lon, oknib_lat, color='yellow', marker='*', s=100, edgecolor='black', label="OKNIB")
    ax.scatter(lis_lon, lis_lat, color='green', marker='^', s=100, edgecolor='black', label="LPPT")

    ax.set_xlim(-10.0, -8.0)
    ax.set_ylim(38.4, 39.4)

    ctx.add_basemap(ax, crs=gdf.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)

    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title(f"Trajectory for Go-Around: {callsign}")
    ax.legend()
    ax.grid(True)

    plt.tight_layout()
    plt.show()

#### feature engineering part 2:

In [ ]:
#cut dataset (11NM to when a ga starts or before a normal flight lands, all the flights without info before a ga are taken out)

import pandas as pd

#function: cuts the flight data right before a go around starts since that data is not necessary 
def go_arounds_cut(group, min_gain=50, min_points=6, tolerance=1):
    altitudes = group["Altitude"].values
    for i in range(len(altitudes) - min_points):
        gain = altitudes[i + min_points] - altitudes[i]
        climb = all(altitudes[i + j + 1] >= altitudes[i + j] - tolerance 
                    for j in range(min_points - 1))

        if climb and gain >= min_gain:
            cut_index = i
            return group.iloc[:cut_index+1]  
    return group

final_dataset = []
df["Distance_NM"] = df["Distance"] / 1852.0  

for landing_index, data in df.groupby("Landing_Index"):
    label = data["Go_Around_Label"].iat[0]
    data = data[data["Distance_NM"] <= 11.0].copy()
    if label == 1:
        final_data = go_arounds_cut(data)
        if len(final_data) < 5:
            continue
    else:  
        final_data = data[data["Distance_NM"] >= 0.25].copy()
        #lppt elevation = 108m, put it a bit higher to avoid having info about the plane on the ground
        touchdown_idx = final_data[final_data["Altitude"] <= 150].index.min() 
        if pd.notna(touchdown_idx):
            final_data = final_data.loc[:touchdown_idx]

    final_data = final_data.dropna(subset=["Altitude", "Distance_NM"])
    if len(final_data) < 5:
        continue

    final_data = final_data.copy()
    final_data["Landing_Index"] = landing_index
    final_data["Go_Around_Label"] = label
    final_dataset.append(final_data)

full_df = pd.concat(final_dataset, ignore_index=True)

In [ ]:
print("Number of unique landings:", full_df["Landing_Index"].nunique())
go_around_count = (full_df.groupby("Landing_Index")["Go_Around_Label"].first().value_counts())
fraction = (go_around_count.get(1, 0)/(full_df["Landing_Index"].nunique()))*100

print("Number of normal landings (0):", go_around_count.get(0, 0))
print("Number of go-arounds (1):", go_around_count.get(1, 0))
print("Go-around %: ", fraction)

In [ ]:
#check to confirm values
features = ["Latitude", "Longitude", "Speed", "Vertical Rate", "Heading", "Altitude"]

n_features = len(features)  
model_df = full_df

for feature in features:
    if feature in model_df.columns:
        values = model_df[feature].dropna() 
        mean = values.mean()
        std = values.std()
        min_val = values.min()
        max_val = values.max()
        print(f"{feature}: mean={mean:.2f}, std={std:.2f}, min={min_val:.2f}, max={max_val:.2f}")
    else:
        print(f"{feature}: not found in model_df")

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
import contextily as ctx

oknib_lon, oknib_lat = -9.17861111, 38.70222222
lis_lon, lis_lat = -9.134167, 38.774167
ist_lon, ist_lat = -9.137961014549424, 38.73990015557909

go_around_df = full_df[full_df['Go_Around_Label'] == 1]
unique_index = go_around_df['Landing_Index'].unique()[:15]  

for callsign in unique_index:
    flight_data = go_around_df[go_around_df['Landing_Index'] == callsign]

    if flight_data.empty:
        continue

    fig, axes = plt.subplots(2, 1, figsize=(10, 8))

    axes[0].plot(flight_data['Time_Sec'], flight_data['Altitude'], 'b-', label="Altitude")
    axes[0].scatter(flight_data['Time_Sec'], flight_data['Altitude'], c='blue', s=5)
    axes[0].set_xlabel('Time')
    #axes[0].set_xlim(0, 1200)
    axes[0].set_ylabel('Altitude (m)')
    axes[0].set_ylim(0, )
    axes[0].grid(True)
    axes[0].legend()

    axes[1].plot(flight_data['Time_Sec'], flight_data['Speed'], 'r-', label="Velocity")
    axes[1].scatter(flight_data['Time_Sec'], flight_data['Speed'], c='red', s=5)
    axes[1].set_xlabel('Time')
    axes[1].set_ylabel('Velocity (m/s)')
    axes[1].set_ylim(0, 200)
    axes[1].grid(True)
    axes[1].legend()

    plt.suptitle(f"Flight Data for Go-Around: {callsign}")
    plt.tight_layout()
    plt.show()

    fig, ax = plt.subplots(figsize=(10, 8))

    gdf = gpd.GeoDataFrame(
        geometry=[Point(lon, lat) for lon, lat in zip(flight_data['Longitude'], flight_data['Latitude'])],
        crs="EPSG:4326"
    )

    gdf.plot(ax=ax, color='blue', linewidth=1, marker='o', markersize=5, label="Trajectory")

    ax.scatter(ist_lon, ist_lat, color='red', s=100, edgecolor='black', label="IST")
    ax.scatter(oknib_lon, oknib_lat, color='yellow', marker='*', s=100, edgecolor='black', label="OKNIB")
    ax.scatter(lis_lon, lis_lat, color='green', marker='^', s=100, edgecolor='black', label="LPPT")

    ax.set_xlim(-10.0, -8.0)
    ax.set_ylim(38.4, 39.4)

    ctx.add_basemap(ax, crs=gdf.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)

    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title(f"Trajectory for Go-Around: {callsign}")
    ax.legend()
    ax.grid(True)

    plt.tight_layout()
    plt.show()

In [ ]:
#calculate new flight features (glideslope, localizer deviation and energy)

import math 
from haversine import haversine, Unit
from geopy.distance import geodesic

def calc_glideslope(altitude, latitude, longitude, runway):
    plane = (latitude, longitude)
    elevation = 108
    if runway == '02':
        runway_points = (38.76638889,-9.14388889)
    else:
        runway_points = (38.79222222,-9.13000000)
    distance = haversine(plane, runway_points, unit=Unit.METERS)
    if distance == 0: 
        return np.nan
    real_alt = altitude - elevation
    glideslope_rad = math.atan(real_alt / distance)
    glideslope_deg = math.degrees(glideslope_rad)
    if (glideslope_deg > 5 or glideslope_deg < 0):
        return np.nan
    return glideslope_deg

def calc_deviation(latitude, longitude, runway):
    if runway == '02':
        lat_i = 38.76638889 #384559.14 
        lon_i = -9.14388889 #0090838.05
        lat_f = 38.79638889 #384747.32
        lon_f = -9.12777778 #0090740.17
    else:
        lat_i = 38.79222222 #384732.39
        lon_i = -9.13000000 #0090748.17
        lat_f = 38.76555556 #384556.44
        lon_f = -9.14416667 #0090839.49
    
    d_lat = lat_f - lat_i
    d_lon = lon_f - lon_i

    d_lat_aircraft = latitude - lat_i
    d_lon_aircraft = longitude - lon_i

    num = (d_lat*d_lat_aircraft + d_lon*d_lon_aircraft)
    dem = d_lat*d_lat + d_lon*d_lon
    if dem == 0:
        return 0
    
    dev = num/dem
    closest_lon = lon_i + dev * d_lon
    closest_lat= lat_i + dev * d_lat

    deviation = geodesic((latitude, longitude), (closest_lat, closest_lon)).meters
    if deviation > 1000:
        return np.nan
    return deviation

def calc_energy(altitude, velocity):
    g = 10
    energy = altitude + (velocity**2)/(2*g)
    return energy

full_df["Glideslope"] = full_df.apply(lambda row: calc_glideslope(row["Correct Altitude"],row["Latitude"], row["Longitude"], row['Runway']), axis=1)
full_df["Deviation"] = full_df.apply(lambda row: calc_deviation(row["Latitude"], row["Longitude"], row['Runway']), axis=1)
full_df["Energy"] = full_df.apply(lambda row: calc_energy(row["Correct Altitude"], row["Speed"]), axis=1)

In [ ]:
#check to confirm the values

features = ["Glideslope", "Deviation", "Energy", "Distance"]

n_features = len(features)  
model_df = full_df

for feature in features:
    if feature in model_df.columns:
        values = model_df[feature].dropna() 
        mean = values.mean()
        std = values.std()
        min_val = values.min()
        max_val = values.max()
        print(f"{feature}: mean={mean:.2f}, std={std:.2f}, min={min_val:.2f}, max={max_val:.2f}")
    else:
        print(f"{feature}: not found in model_df")

In [ ]:
#calculate go around clustering effects (ga count hourly, ga last time, ga before)

##########################################################################################################################################################
#how it works:

#1. organize the ga data info (date, landing index, time in seconds, distance and timestamp when it starts) and flight data info (date, landing index, icao, callsign, time, time in seconds and go around label)

#2. for each row in the flight data info
    #ga before = variable previous ga that says if the previous flight index was a ga or not
    #if time since a ga happened = none 
        #ga last time = 1440 min 
    #else: 
        #ga last time = time diff between the time since a ga happened and the flight time
    #ga hourly = sum of all go arounds that happened in the last hour 
    #update the flight info and the ga variables in case the next flight is a go around (it is important to be careful in order to leaking future data)

#4. add go around clustering effects features to dataset
##########################################################################################################################################################

import pandas as pd

df = full_df.copy()
df["Date"] = pd.to_datetime(df["Date"])
ga_df = df[df["Go_Around_Label"] == 1].copy()

ga_start = (ga_df.groupby(["Date", "Landing_Index"]).last().reset_index()[["Date", "Landing_Index", "Time_Sec", "Distance"]]
.rename(columns={"Time_Sec": "GA_Start_Time_Sec", "Distance": "GA_Start_Distance"}))

ga_start["GA_Start_Timestamp"] = pd.to_datetime(ga_start["Date"].astype(str)) + pd.to_timedelta(ga_start["GA_Start_Time_Sec"], unit="s")

ga_info = ga_start.set_index(["Date", "Landing_Index"])["GA_Start_Timestamp"].to_dict()

flight_info = (df.groupby(["Date", "Landing_Index"]).agg({"ICAO": "max", "Callsign": "max", "Time": "min", "Time_Sec": "min", "Go_Around_Label": "max"}).reset_index())
flight_info["Timestamp"] = pd.to_datetime(flight_info["Date"].astype(str) + " " + flight_info["Time"].astype(str))
flight_info = flight_info.sort_values("Timestamp").reset_index(drop=True)

results = []
previous_ga = 0
last_ga_time = None
ga_history = []

for idx, row in flight_info.iterrows():
    current_time = row["Timestamp"]

    ga_before = previous_ga
    if last_ga_time is None:
        ga_lasttime = 1440
    else:
        delta_minutes = (current_time - last_ga_time).total_seconds() / 60.0
        ga_lasttime = delta_minutes if delta_minutes <= 1440 else 1440

    past_hour = current_time - pd.Timedelta(hours=1)
    ga_hourly = sum(1 for t in ga_history if past_hour <= t < current_time)

    results.append({
        "Date": row["Date"],
        "Landing_Index": row["Landing_Index"],
        "GA_before": ga_before,
        "GA_lasttime": ga_lasttime,
        "GA_hourly": ga_hourly
    })

    if idx + 1 < len(flight_info):
        next_row = flight_info.iloc[idx + 1]
        next_info = f"{next_row['Date']} | Landing_Index: {next_row['Landing_Index']} | Time: {next_row['Time']} | GA_Label: {next_row['Go_Around_Label']}"
    else:
        next_info = "None"

    print(f"Flight {idx}: Date: {row['Date']} | Landing_Index: {row['Landing_Index']} | Time: {row['Time']} | GA_Label: {row['Go_Around_Label']}")
    print(f"GA_before: {ga_before} | GA_lasttime: {ga_lasttime:.1f} min | GA_hourly: {ga_hourly}")
    print(f"Next flight -> {next_info}\n")

    if row["Go_Around_Label"] == 1:
        ga_start_time = ga_info.get((row["Date"], row["Landing_Index"]), None)
        if ga_start_time is not None and pd.notna(ga_start_time) and ga_start_time <= current_time:
            last_ga_time = ga_start_time
            ga_history.append(last_ga_time)
        else:
            if last_ga_time is None or current_time > last_ga_time:
                last_ga_time = current_time
                ga_history.append(last_ga_time)
        previous_ga = 1
    else:
        previous_ga = 0

ga_final_df = pd.DataFrame(results)
final_df = full_df.merge(ga_final_df, on="Landing_Index", how="left")
final_df = final_df.drop(columns=["Date_y"], errors="ignore")
final_df = final_df.rename(columns={"Date_x": "Date"})

In [ ]:
#check to confirm values
features = ["GA_before", "GA_lasttime", "GA_hourly"]

n_features = len(features)  
model_df = final_df

for feature in features:
    if feature in model_df.columns:
        values = model_df[feature].dropna() 
        mean = values.mean()
        std = values.std()
        min_val = values.min()
        max_val = values.max()
        print(f"{feature}: mean={mean:.2f}, std={std:.2f}, min={min_val:.2f}, max={max_val:.2f}")
    else:
        print(f"{feature}: not found in model_df")

In [ ]:
#calculate in trail relationship features (los, speed and altitude difference, lead aircraft)

##########################################################################################################################################################
#how it works:

#1. organize the flight data info (landing index) per timestamp and runway

#2. for each runway group, create a list with pairs of consecutive flights and calculate the time difference
    #if time_diff < 10:
        #trailing - leading pair exists
    #else:
        #skip

#3. for each pair
    #find the timestamp when the trailing aircraft was exactly 5 NM from the runway (time_5nm)
    #interpolate the latitude, longitude, altitude and speed of the trailing and leading aircraft at time_5nm
    #calcultate the separation between the leading and trailing aircraft (St)
    #calculate the altitude and speed difference
    #calculate the loss of separation (max(0, Sm-St))

#4. add the in trail relationships features to dataset
##########################################################################################################################################################

import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from haversine import haversine, Unit

#function: interpolates the flight's features 
def interpolate_flight(flight_df, target_time, features=["Latitude", "Longitude", "Altitude", "Speed"]):
    flight_df = flight_df.sort_values("Timestamp")
    results = {}
    for feature in features:
        x = flight_df["Timestamp"].astype(np.int64) / 1e9
        y = flight_df[feature]
        if len(x) < 2 or x.nunique() < 2:
            results[feature] = np.nan
            continue
        f_interp = interp1d(x, y, kind='linear', bounds_error=False, fill_value="extrapolate")
        results[feature] = f_interp(target_time.timestamp())
    return results

df = final_df.copy()

df["Timestamp"] = pd.to_datetime(df["Date"].astype(str) + " " + df["Time"].astype(str))
df = df.sort_values(by=["Runway", "Timestamp"])

flight_info = (df.groupby(["Runway", "Landing_Index"])["Timestamp"].max().reset_index().sort_values(by="Timestamp"))

pairs = []
for rw in flight_info["Runway"].unique():
    runway_df = flight_info[flight_info["Runway"] == rw]
    for i in range(1, len(runway_df)):
        trail = runway_df.iloc[i - 1]
        lead = runway_df.iloc[i]
        time_diff = (trail["Timestamp"] - lead["Timestamp"]).total_seconds() / 60
        if time_diff < 10:
            pairs.append((lead["Landing_Index"], trail["Landing_Index"]))

df["Distance_NM"] = df["Distance"] / 1852
results = []

for lead_idx, trail_idx in pairs:
    lead_df = df[df["Landing_Index"] == lead_idx].sort_values("Distance_NM", ascending=False)
    trail_df = df[df["Landing_Index"] == trail_idx].sort_values("Distance_NM", ascending=False)

    if not (trail_df["Distance_NM"].min() <= 5.0 <= trail_df["Distance_NM"].max()):
        continue  
    
    trail_distance = trail_df["Distance_NM"]
    trail_time = trail_df["Timestamp"].astype(np.int64) / 1e9
    f_time = interp1d(trail_distance, trail_time, kind='linear', bounds_error=False, fill_value=np.nan)
    t_5nm = f_time(5.0)
    if np.isnan(t_5nm):
        continue
    time_5nm = pd.to_datetime(t_5nm, unit="s")

    if time_5nm < lead_df["Timestamp"].min() or time_5nm > lead_df["Timestamp"].max():
        continue

    trail_info = interpolate_flight(trail_df, time_5nm, features=["Latitude", "Longitude", "Altitude", "Speed"])
    lead_info = interpolate_flight(lead_df,  time_5nm, features=["Latitude", "Longitude", "Altitude", "Speed"])

    if any(pd.isna([trail_info["Latitude"], trail_info["Longitude"], 
                    lead_info["Latitude"], lead_info["Longitude"]])):
        continue

    coordinates_lead = (float(lead_info["Latitude"]), float(lead_info["Longitude"]))
    coordinates_trail = (float(trail_info["Latitude"]), float(trail_info["Longitude"]))
    St = haversine(coordinates_lead, coordinates_trail, unit=Unit.NAUTICAL_MILES)

    SpeedDiff_lt = lead_info["Speed"] - trail_info["Speed"]
    SpeedDiff_tl = trail_info["Speed"] - lead_info["Speed"]
    AltDiff_lt = lead_info["Altitude"] - trail_info["Altitude"]
    AltDiff_tl = trail_info["Altitude"] - lead_info["Altitude"]

    Sm = 4.0 
    LOS = max(0, Sm - St)

    results.append({
        "Lead_Index": lead_idx,
        "Trail_Index": trail_idx,
        "Lead_Alt": lead_info["Altitude"],
        "Trail_Alt": trail_info["Altitude"], 
        "Lead_Speed": lead_info["Speed"],
        "Trail_Speed": trail_info["Speed"],
        "Separation": St,
        "LOS": LOS,
        "SpeedDiff_lt": SpeedDiff_lt,
        "AltDiff_lt": AltDiff_lt,
        "SpeedDiff_tl": SpeedDiff_tl,
        "AltDiff_tl": AltDiff_tl,
        "Time": time_5nm,
    })

df_separation = pd.DataFrame(results)
df_filtered = df_separation[df_separation["LOS"] <= 5].copy().reset_index(drop=True)

trailing_with_lead = set(df_filtered["Trail_Index"])
final_df["Lead_Aircraft"] = final_df["Landing_Index"].apply(lambda x: 1 if x in trailing_with_lead else 0)

df_filtered_renamed = df_filtered.rename(columns={
    "Trail_Index": "Landing_Index",
    "SpeedDiff_lt": "Speed_Diff_lt",
    "AltDiff_lt": "Alt_Diff_lt",
    "SpeedDiff_tl": "Speed_Diff_tl",
    "AltDiff_tl": "Alt_Diff_tl"
})

final_df = final_df.merge(df_filtered_renamed[["Landing_Index", "LOS", "Alt_Diff_lt", "Speed_Diff_lt", "Alt_Diff_tl", "Speed_Diff_tl"]], on="Landing_Index", how="left")

In [ ]:
#plot: trailing vs leading aircraft altitude and speed difference and los

import matplotlib.pyplot as plt

for trail_idx, df in df_separation.groupby("Trail_Index"):

    print(f"{trail_idx}")
    St = df["Separation"].iloc[0]
    Sm = 4.0  
    LOS = df["LOS"].iloc[0]
    lead_alt = df["Lead_Alt"].iloc[0]
    trail_alt = df["Trail_Alt"].iloc[0]
    alt_diff = df["AltDiff_lt"].iloc[0]
    lead_speed = df["Lead_Speed"].iloc[0]
    trail_speed = df["Trail_Speed"].iloc[0]
    speed_diff = df["SpeedDiff_lt"].iloc[0]

    #Altitude
    plt.figure(figsize=(7,6))
    plt.style.use("seaborn-v0_8-darkgrid")

    plt.scatter(0, lead_alt, color="blue", s=200, marker=">", label="Leading Aircraft")
    plt.scatter(St, trail_alt, color="red", s=200, marker="*", label="Trailing Aircraft")

    plt.axvline(Sm, color="gray", linestyle="--", label=f"Sm = {Sm:.1f} NM")
    plt.annotate(
      "", 
      xy=(St/2, trail_alt-10), 
      xytext=(St/2, lead_alt-10), 
      arrowprops=dict(arrowstyle="<->", color="black", lw=1.5))
    plt.text(St/2+0.1, (lead_alt+trail_alt)/2, "Δh", va="center", fontsize=12)

    if LOS > 0:
        plt.annotate(
            "", 
            xy=(Sm, lead_alt), 
            xytext=(St, lead_alt), 
            arrowprops=dict(arrowstyle="<->", color="black", lw=1.5))
        plt.text((Sm+St)/2, lead_alt-20, f"LOS = {LOS:.2f} NM", ha="center", va="top", fontsize=12) 

    plt.xlim(-0.5, 4.5)  
    plt.ylim(0, 1000) 
    plt.xlabel("Separation (NM)", fontsize=18)
    plt.ylabel("Altitude (m)", fontsize=18)
    plt.legend(fontsize=14)
    plt.xticks(fontsize=15, rotation=0)
    plt.yticks(fontsize=15)
    plt.legend()
    ax = plt.gca()
    for spine in ax.spines.values():
        spine.set_linewidth(1.5)
        spine.set_color("black")
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.show()

    #Speed
    plt.figure(figsize=(7,6))

    plt.scatter(0, lead_speed, color="blue", s=200, marker=">", label="Leading Aircraft")
    plt.scatter(St, trail_speed, color="red", s=200, marker="*", label="Trailing Aircraft")

    plt.axvline(Sm, color="gray", linestyle="--", label=f"Sm = {Sm:.1f} NM")
    plt.annotate(
      "", 
      xy=(St/2, trail_speed), 
      xytext=(St/2, lead_speed), 
      arrowprops=dict(arrowstyle="<->", color="black", lw=1.5))
    plt.text(St/2+0.1, (lead_speed+trail_speed)/2, "Δv", va="center", fontsize=12)

    if LOS > 0:
        plt.annotate(
            "", 
            xy=(Sm, lead_speed), 
            xytext=(St, lead_speed), 
            arrowprops=dict(arrowstyle="<->", color="black", lw=1.5))
        plt.text((Sm+St)/2, lead_speed-3, f"LOS = {LOS:.2f} NM", ha="center", va="top", fontsize=12) 

    plt.xlim(-0.5, 4.5)  
    plt.ylim(0, 100) 
    plt.xlabel("Separation (NM)", fontsize=18)
    plt.ylabel("Speed (m/s)", fontsize=18)
    plt.legend(fontsize=13)
    plt.xticks(fontsize=15, rotation=0)
    plt.yticks(fontsize=15)
    plt.legend()
    ax = plt.gca()
    for spine in ax.spines.values():
        spine.set_linewidth(1.5)
        spine.set_color("black")
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.show()

In [ ]:
#check all the values 

features = ["Latitude", "Longitude", "Speed", "Vertical Rate", "Heading", "Correct Altitude", "Glideslope", "Deviation", "Energy", "Wind_Speed_M",
            "Wind_Direction_M", "Visibility_M", "G_M", "TS_M", "FG_M", "RA_M", "WS_M", "Wind_Dir_Change", "Wind_Dir_Change_M", "Wind_Dir_Sector_M"
            "Wind_CompU_E", "Wind_CompV_E", "Wind_Gust_E", "Wind_Shear_E", "CBHA_E", "Visibility_E", "C_1000_E", "C_500_E", "LOS", "Alt_Diff_lt", "Speed_Diff_lt",
            "Alt_Diff_tl", "Speed_Diff_tl", "GA_before", "GA_lasttime", "GA_hourly" ]

n_features = len(features)  
model_df = final_df

for feature in features:
    if feature in model_df.columns:
        values = model_df[feature].dropna() 
        mean = values.mean()
        std = values.std()
        min_val = values.min()
        max_val = values.max()
        print(f"{feature}: mean={mean:.2f}, std={std:.2f}, min={min_val:.2f}, max={max_val:.2f}")
    else:
        print(f"{feature}: not found in model_df")

In [ ]:
final_df = final_df.drop(columns=["LOS_x", "Alt_Diff_lt_x", "Speed_Diff_lt_x", "Alt_Diff_tl_x", "Speed_Diff_tl_x", "LOS_y", "Alt_Diff_lt_y", "Speed_Diff_lt_y", "Alt_Diff_tl_y", "Speed_Diff_tl_y"], errors="ignore")

In [ ]:
final_df.to_csv("april24_gas.csv", index=False)
final_df = pd.read_csv("april24_gas.csv", dtype={"ICAO": str})
final_df